In [13]:
% reload_ext autoreload
% autoreload 2
% aimport cellCnn

UsageError: Line magic function `%` not found.


In [14]:
import pickle
import cellCnn
import importlib

import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt

from cellCnn.ms.utils.helpers import get_min_mean_from_clusters, get_chunks
from cellCnn.utils import mkdir_p
from cellCnn.plotting import plot_results
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results, get_selected_cells, get_data
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df

import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from plotting import plot_filters, discriminative_filters

def reload_modules():
    import cellCnn
    import cellCnn.utils

    importlib.reload(cellCnn.ms.utils.helpers)
    importlib.reload(cellCnn.utils)
    importlib.reload(cellCnn.model)
    importlib.reload(cellCnn)
    import cellCnn
    import cellCnn.utils
    pass

In [15]:
#%reset

In [16]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input.nosync'
outdir = 'class_cd8'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840  # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}
outfolder = 'mtl_1'
mkdir_p(outfolder)

In [17]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

FileNotFoundError: [Errno 2] No such file or directory: 'data/input.nosync/cohort_denoised_clustered_diagnosis_patients.csv'

In [ ]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

In [11]:
#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 


NameError: name 'rrms_patients2df' is not defined

In [12]:
### desease states 1 = RRMS and 0 = NINDC
batch_size_dict = dict()
selection_pool_rrms_cd8 = get_chunks_from_df(rrms_patients2df,
                                             freq_df=rrms_patients_freq,
                                             desease_state=1)
selection_pool_nindc_cd8 = get_chunks_from_df(nindc_patients2df,
                                              freq_df=nindc_patients_freq,
                                              desease_state=0)

# make sure list are equally long:
if len(selection_pool_rrms_cd8) > len(selection_pool_nindc_cd8):
    selection_pool_rrms_cd8 = selection_pool_rrms_cd8[:len(selection_pool_nindc_cd8)]
elif len(selection_pool_rrms_cd8) < len(selection_pool_nindc_cd8):
    selection_pool_nindc_cd8 = selection_pool_nindc_cd8[:len(selection_pool_rrms_cd8)]

all_chunks = selection_pool_rrms_cd8 + selection_pool_nindc_cd8
np.random.shuffle(all_chunks)

X = [selection[0].to_numpy() for selection in all_chunks]
freqs = [selection[1] for selection in all_chunks]
Y = [selection[2] for selection in all_chunks]

print('DONE: batches created')

NameError: name 'rrms_patients2df' is not defined

In [8]:
# Default values were used for most CellCNN hyperparameters,
# except for the following: ncell = 2,000; scale = false;
# maxpool percentages = (0.5, 1.0, 2.5, 5.0, 10.0). T
reload_modules()
# for regression task stratified is wrong since there are no classes
freq_idx = 3
cd8 = [series[freq_idx] for series in freqs]
X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
    X, cd8, Y,
    train_perc=train_perc,
    test_perc=test_perc,
    valid_perc=0.5)

# pay attention to put in phenotype first and then freq...
model = get_fitted_model(X_train, X_valid, [y_train, cd8_train], [y_valid, cd8_valid, ],
                         nsubset=100, ncell=2000,
                         quant_normed=True, scale=False,
                         nfilters=[3, 7, 20, 35], max_epochs=20,
                         learning_rate=None, nrun=15,
                         per_sample=True, regression=False,
                         maxpool_percentages = [0.5, 1.0, 2.5, 5.0, 10.0],
                         outdir=outdir, verbose=True, mtl_tasks=2)
results = model.results
pickle.dump(results, open(os.path.join(outdir, 'results.pkl'), 'wb'))
save_results(results, outdir, cytokines)
print('DONE building models')

NameError: name 'split_test_train_valid' is not defined

In [26]:
# exported_filter_weights: consensus filtzer from results['selected_filters'], filters_all from results['clustering_result']

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [16]:
plotdir = os.path.join(outdir, 'plots')

# consensus_filter_weights, best_net, clustered_filter_weights
plot_filters(results, cytokines, os.path.join(plotdir, 'filter_plots'))

# filter response difference.pdf -> only for consensus filters
_v = discriminative_filters(results, os.path.join(plotdir, 'filter_plots_discriminative'),
                            filter_diff_thres=0.2, show_filters=True)
print('done')

done


In [17]:
# just took default values
# colors in plots are coming from the sample input (here the X_train)
filter_info = plot_results(results, X_train, y_train,
                           cytokines, os.path.join(plotdir, 'training_plots'),
                           filter_diff_thres=0.2,
                           filter_response_thres=0,
                           stat_test='mannwhitneyu',
                           group_a='RRMS', group_b='NINDC',
                           group_names=['RRMS', 'NINDC'],
                           tsne_ncell=1000,
                           regression=False,
                           show_filters=True)
print('done')

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/seaborn/categorical.py:1296: UserWarning: 54.2% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/seaborn/categorical.py:1296: UserWarning: 45.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWa

done


In [19]:
_v = plot_results(results, X_valid, y_valid,
                  cytokines, os.path.join(plotdir, 'validation_plots'),
                  filter_diff_thres=0.2,
                  filter_response_thres=0,
                  stat_test='mannwhitneyu',
                  group_a='RRMS', group_b='NINDC',
                  group_names=['Test_a', 'test_b'],
                  tsne_ncell=1000,
                  regression=False,
                  show_filters=True)
csv_dir = os.path.join(outdir, 'selected_cells')
mkdir_p(csv_dir)
nfilter = len(filter_info)

# args.fcs = des gibt mir die fcs folder ich hol mir in sample names also die verschiedenen fcs filenames raus...
# ich arbeite nur mit einem fcs file  (ich nutze ja nur den cohort), drum verteil ich den name jetzt einfach

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [5]:
sample_names = ['selected_cells/cohort']
df2 = pd.read_csv(os.path.join(indir, infile), index_col=0)
df2 = df2.drop_duplicates()
test = df2.iloc[:, :len(cytokines)]
samples = [test]
# for each sample
for x, x_name in zip(samples, sample_names):
    flags = np.zeros((x.shape[0], 2 * nfilter))
    columns = []
    # for each filter
    for i, (filter_idx, thres) in enumerate(filter_info):
        flags[:, 2 * i:2 * (i + 1)] = get_selected_cells(
            results['selected_filters'][filter_idx], x, results['scaler'], thres, True)
        columns += ['filter_%d_continuous' % filter_idx, 'filter_%d_binary' % filter_idx]
    df = pd.DataFrame(flags, columns=columns)
    df.to_csv(os.path.join(outdir, x_name + '_selected_cells.csv'), index=False)

print('done')


NameError: name 'indir' is not defined

In [21]:
reload_modules()
######################################################################
# what predict does is: Iterating over the best 3 nets and return an array per net with the predictions for all samples!
stats_file = open(f"{outdir}/stats_class_{model.ncell}_{model.nsubset}.txt", "w+")
print('Write stats to: \n')
print(stats_file.name)

stats_file.write(f"CellCNN Model {outdir}  with ncells: {model.ncell} and subsets: {model.nsubset}\n")
stats_file.write(f"Best model was {model.results['best_model_index']} \n")

test_pred = model.predict(X_test, mtl_inputs=[y_test, cd8_test])
train_pred = model.predict(X_train, mtl_inputs=[y_train, cd8_train])
valid_pred = model.predict(X_valid, mtl_inputs=[y_valid, cd8_valid])
#print_regression_model_stats(test_pred, b_test)
test_pred_abs = [1 if pred[0] > pred[1] else 0 for pred in test_pred[0]]
train_pred_abs = [1 if pred[0] > pred[1] else 0 for pred in train_pred[0]]
valid_pred_abs = [1 if pred[0] > pred[1] else 0 for pred in valid_pred[0]]

acc_test = accuracy_score(list(y_test), test_pred_abs)
acc_train = accuracy_score(list(y_train), train_pred_abs)
acc_valid = accuracy_score(list(y_valid), valid_pred_abs)
stats_file.write('Desease: \n')
stats_file.write(f'Acc y test {acc_test}\n')
stats_file.write(f'Acc y train {acc_train}\n')
stats_file.write(f'Acc y valid {acc_valid}\n')

mse_test_cd8 = mean_squared_error(list(cd8_test), test_pred[1])
mse_train_cd8 = mean_squared_error(list(cd8_train), train_pred[1])
mse_valid_cd8 = mean_squared_error(list(cd8_valid), valid_pred[1])
stats_file.write('Cd8: \n')
stats_file.write(f'MSE y test {mse_test_cd8}\n')
stats_file.write(f'MSE y train {mse_train_cd8}\n')
stats_file.write(f'MSE y valid {mse_valid_cd8}\n')
stats_file.write('\n')
stats_file.close()
print('DONE')

Write stats to: 

class_cd8/stats_class_2000_100.txt
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs
DONE
